# PROJECT-3. Исследование поведения пользователей
#### Работу выполнила: Машковцева Дарья

Задание: 
1. Исследовать зависимость вероятности оплаты от выбранного пользователем уровня сложности.
2. Существует ли разница во времени между событиями регистрации и оплаты для разных групп пользователей с разным уровнем сложности?

! Расчеты выполняются на основе данных пользователей, которые зарегистрировались в 2018 году.

Краткий аглоритм работы:
1. Получите данные из файла.
2. Произведите обзор данных и преобразование данных, если оно необходимо.
3. Выделите группы пользователей (по аналогии с группами, которые мы выделяли, когда находили зависимость между фактом прохождения обучения и оплатой).
4. Рассчитайте для каждой группы процент оплат (это мы тоже уже делали).
5. Для каждой группы сделайте по два датафрейма: один — с событиями выбора уровня сложности, другой — с событиями оплаты.
6. Объедините датафреймы в рамках одной группы и найдите разницу во времени между событиями (по аналогии с расчётом, который мы уже производили, когда находили время между событиями для всех пользователей).
7. Рассчитайте среднее время между событиями.

In [ ]:
# Импорт библиотек и выгрузка данных

import pandas as pd
import psycopg2
import psycopg2.extras 
import numpy as np

def getEventsData():
    query = '''SELECT e.* FROM case8.events e
    WHERE e.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='lms.skillfactory.ru' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

def getPurchaseData():
    query = '''SELECT p.* FROM case8.purchase p
    WHERE p.user_id in
    (SELECT DISTINCT(user_id) 
    FROM case8.events 
    WHERE event_type = 'registration' 
    AND start_time >= '2017-01-01'
    AND start_time < '2018-01-01')
    '''.format()
    conn = psycopg2.connect("dbname='skillfactory' user='skillfactory' host='lms.skillfactory.ru' password='cCkxxLVrDE8EbvjueeMedPKt' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
    dict_cur.execute(query)
    rows = dict_cur.fetchall()
    data = []
    for row in rows:
        data.append(dict(row))
    return data

events_df = pd.DataFrame(getEventsData())
purchase_df = pd.DataFrame(getPurchaseData())